# Bond Valuation

In [12]:
def bond_value(face_value, coupon_rate, years_to_maturity, ytm, frequency='annual'):
    """
    Calculate the present value of a bond.

    Parameters:
    face_value (float): The face value of the bond.
    coupon_rate (float): The annual coupon interest rate (as a decimal, e.g., 0.05 for 5%).
    years_to_maturity (int): Number of years until the bond matures.
    ytm (float): Yield to maturity (as a decimal, e.g., 0.04 for 4%).
    frequency (str): 'annual' for annual coupon payments or 'semi-annual' for semi-annual payments.

    Returns:
    float: Present value of the bond.
    """
    if frequency not in ['annual', 'semi-annual']:
        raise ValueError("Frequency must be 'annual' or 'semi-annual'.")

    # Determine number of periods and rate per period
    if frequency == 'annual':
        periods_per_year = 1
    else:
        periods_per_year = 2

    total_periods = years_to_maturity * periods_per_year
    rate_per_period = ytm / periods_per_year
    coupon_payment = (coupon_rate * face_value) / periods_per_year

    # Calculate present value of the coupon payments
    coupon_pv = sum(coupon_payment / (1 + rate_per_period) ** t for t in range(1, total_periods + 1))

    # Calculate present value of the face value (lump sum at maturity)
    face_value_pv = face_value / (1 + rate_per_period) ** total_periods

    # Total present value of the bond
    bond_price = coupon_pv + face_value_pv
    return bond_price

# Example usage
face_value = 1000  # Face value of the bond
coupon_rate = 0.12  # Coupon interest rate (5%)
years_to_maturity = 15  # Years to maturity
ytm = 0.16  # Yield to maturity (4%)
frequency = 'semi-annual'  # Payment frequency: 'annual' or 'semi-annual'

bond_price = bond_value(face_value, coupon_rate, years_to_maturity, ytm, frequency)
print(f"The bond price is: ${bond_price:.3f}")

The bond price is: $774.844


In [ ]:
from scipy.optimize import newton

def calculate_ytm(bond_price, face_value, coupon_rate, years_to_maturity, frequency='annual', guess=0.05):
    """
    Calculate the yield to maturity of a bond.

    Parameters:
    bond_price (float): The current price of the bond.
    face_value (float): The face value of the bond.
    coupon_rate (float): The annual coupon interest rate (as a decimal).
    years_to_maturity (int): Number of years until the bond matures.
    frequency (str): 'annual' for annual coupon payments or 'semi-annual' for semi-annual payments.
    guess (float): Initial guess for YTM (as a decimal).

    Returns:
    float: Yield to maturity (as a decimal).
    """
    func = lambda ytm: bond_price - bond_value(face_value, coupon_rate, years_to_maturity, ytm, frequency)
    ytm = newton(func, guess)
    return ytm

# Example usage
bond_price = 774.85  # Current bond price
face_value = 1000  # Face value of the bond
coupon_rate = 0.12  # Coupon interest rate (5%)
years_to_maturity = 15  # Years to maturity
frequency = 'semi-annual'  # Payment frequency: 'annual' or 'semi-annual'

ytm = calculate_ytm(bond_price, face_value, coupon_rate, years_to_maturity, frequency)
print(f"The ytm is: {ytm:.4%}")

The yield to maturity is: 15.9999%


In [ ]:
def calculate_realized_yield(
        face_value, coupon_rate, current_ytm, years_to_maturity, 
        holding_period, future_ytm, frequency='annual'):
    """
    Calculate the realized yield of holding a bond for a given period.

    Parameters:
    face_value (float): The face value of the bond.
    coupon_rate (float): The annual coupon interest rate (as a decimal).
    current_ytm (float): Initial yield to maturity (as a decimal).
    years_to_maturity (int): Original years to maturity when bond was purchased.
    holding_period (int): Number of years the bond is held.
    future_ytm (float): Yield to maturity when the bond is sold (as a decimal).
    frequency (str): 'annual' for annual coupon payments or 'semi-annual' for semi-annual payments.

    Returns:
    float: Realized yield (as a decimal).
    """
    if frequency not in ['annual', 'semi-annual']:
        raise ValueError("Frequency must be 'annual' or 'semi-annual'.")

    # Calculate the initial bond price based on the current YTM
    initial_bond_price = bond_value(face_value, coupon_rate, years_to_maturity, current_ytm, frequency)

    # Calculate the future bond price when sold after the holding period
    remaining_years_to_maturity = years_to_maturity - holding_period
    future_bond_price_at_sale = bond_value(face_value, coupon_rate, remaining_years_to_maturity, future_ytm, frequency)

    # Calculate the total cash flows from holding the bond
    if frequency == 'annual':
        periods_per_year = 1
    else:
        periods_per_year = 2
    holding_periods = holding_period * periods_per_year
    counpon_payment = (coupon_rate * face_value) / periods_per_year
    func = lambda r_ytm: initial_bond_price - sum(counpon_payment / (1 + r_ytm / periods_per_year) ** t for t in range(1, holding_periods + 1)) - future_bond_price_at_sale / (1 + r_ytm / periods_per_year) ** holding_periods
    realized_yield = newton(func, current_ytm)

    return realized_yield

# Example usage
face_value = 1000  # Face value of the bond
coupon_rate = 0.05  # Coupon interest rate (5%)
current_ytm = 0.08  # Current yield to maturity (4%)
years_to_maturity = 20  # Years to maturity
holding_period = 2  # Number of years holding the bond
future_ytm = 0.07  # Future yield to maturity when selling the bond (4.5%)
frequency = 'semi-annual'  # Payment frequency: 'annual' or 'semi-annual'

realized_yield = calculate_realized_yield(
    face_value, coupon_rate, current_ytm, years_to_maturity, 
    holding_period, future_ytm, frequency)
print(f"The realized yield is: {realized_yield:.4%}")

The realized yield is: 13.1698%


# Stock dividend discount model

In [ ]:
def pv_stock_constant_growth(
        dividend, growth_rate, discount_rate, 
        pay_now = False,
        payment_frequency='annual'
        ):
    """
    Calculate the present value of a stock with constant growth.

    Parameters:
    dividend (float): The dividend payment.
    growth_rate (float): The constant growth rate. YEARLY!!!
    discount_rate (float): The discount rate (required rate of return). APR!!!
    pay_now (bool): Whether the dividend is paid now or at the end of the period.
    payment_frequency (str): 'annual', 'semi-annual', or 'quarter'.

    Returns:
    float: Present value of the stock.
    """
    if payment_frequency not in ['annual', 'semi-annual', 'quarter']:
        raise ValueError("Payment frequency must be 'annual', 'semi-annual', or 'quarter'.")
    if payment_frequency == 'annual':
        periods_per_year = 1
    elif payment_frequency == 'semi-annual':
        periods_per_year = 2
    else:
        periods_per_year = 4
    growth_rate /= periods_per_year
    discount_rate /= periods_per_year
    if pay_now:
        stock_price = dividend * (1 + growth_rate) / (discount_rate - growth_rate)
    else:
        stock_price = dividend / (discount_rate - growth_rate)
    return stock_price

# Example usage
dividend = 0.22  # Dividend payment
growth_rate = 0.001 * 4  # Growth rate: should be yearly!!!!!
discount_rate = 0.16  # Discount rate
pay_now = False  # Dividend is paid now
payment_frequency = 'quarter'  # Payment frequency: 'annual', 'semi-annual', or 'quarter'
stock_price = pv_stock_constant_growth(dividend, growth_rate, discount_rate, pay_now, payment_frequency)
print(f"The stock price is: ${stock_price:.2f}")



The stock price is: $5.64


In [58]:
def pv_stock_varying_growth_rate(
        last_dividend, discount_rate, growth_rates, 
        year_start_infinite_growth
        ):
    """
    Calculate the present value of a stock with varying growth rates.

    Parameters:
    last_dividend (float): The dividend at time 0.
    discount_rate (float): The discount rate for future cash flows.
    growth_rates (list): The growth rates of dividends for each year.
    year_start_infinite_growth (int): The year when dividends grow at a constant rate indefinitely.
    """
    pv = 0
    # Calculate the present value of dividends until the year of infinite growth
    for t, g in enumerate(growth_rates):
        t += 1 # since count from 0, need to add 1 manually
        last_dividend *= 1 + g
        if t < year_start_infinite_growth:
            pv += last_dividend / (1 + discount_rate) ** t
            #print(f"Period {t}: Dividend = ${last_dividend:.3f}", f"(growth {g})", f"PV = ${pv:.3f}")
        else:
            #print(f"Period {t}: Dividend = ${last_dividend:.3f} (infinite growth {g})")
            # Calculate the present value of dividends with infinite growth
            pv += last_dividend / (discount_rate - g) / (1 + discount_rate) ** (t-1)
    return pv

# Example usage
# Normal use case
last_dividend = 1
discount_rate = 0.2
growth_rates = [0.2, 0.15, 0.05]
year_start_infinite_growth = 3
stock_price = pv_stock_varying_growth_rate(last_dividend, discount_rate, growth_rates, year_start_infinite_growth)
print(f"The stock price is: ${stock_price:.2f}")

# Below is an example if we use quatrerly growth rates
growth_rates = [0.032/4] * 5 * 4 + [-0.02/4] 
last_dividend = 0.52 # 
discount_rate = 0.16 / 4
year_start_infinite_growth = 5 * 4 + 1
stock_price = pv_stock_varying_growth_rate(last_dividend, discount_rate, growth_rates, year_start_infinite_growth)
print(f"The stock price is: ${stock_price:.2f}")

growth_rates = [0.032/4] * 5 * 4 + [0] 
last_dividend = 0.52 # 
discount_rate = 0.16 / 4
year_start_infinite_growth = 5 * 4 + 1
stock_price = pv_stock_varying_growth_rate(last_dividend, discount_rate, growth_rates, year_start_infinite_growth)
print(f"The stock price is: ${stock_price:.2f}")

The stock price is: $8.67
The stock price is: $13.77
The stock price is: $14.57


In [46]:
[0.032/4] * 5 + [-0.02]

[0.008, 0.008, 0.008, 0.008, 0.008, -0.02]

# Portfolio Theory

capital allocation line: allocation of a portfolio and risk-free assets,
$$
    E(R_p) = R_f + \frac{E(R_i)-R_f}{\sigma_i}\sigma_p
$$

Where $\sigma_p$ is decided by the market. So what is important is $\frac{E(R_i)-R_f}{\sigma_i}$, which is the sharp ratio.

In [67]:
# calcualte optimal weight allocation for a two-asset portfolio
def optimal_weight_allocation(r1, r2, sigma1, sigma2, rho, rf):
    """
    Calculate the optimal weight allocation for a two-asset portfolio.

    Parameters:
    r1 (float): Expected return of asset 1.
    r2 (float): Expected return of asset 2.
    sigma1 (float): Standard deviation of asset 1.
    sigma2 (float): Standard deviation of asset 2.
    rho (float): Correlation coefficient between asset 1 and asset 2.

    Returns:
    tuple: Optimal weight allocation (weight of asset 1, weight of asset 2).
    """
    rp1 = r1 - rf
    rp2 = r2 - rf
    sigma12 = sigma1 * sigma2 * rho
    w1 = ( sigma2**2 * rp1 - sigma12 * rp2 ) / ( sigma1**2 * rp2 + sigma2**2 * rp1 - sigma12 * (rp1 + rp2) )
    w2 = 1 - w1
    return w1, w2

# use annualized return and standard deviation
r1 = 0.043381
std1 = 0.036666

r2 = 0.096887
std2 = 0.151002
rho = 0.04155
rf = 0.02
w1, w2 = optimal_weight_allocation(r1, r2, std1, std2, rho, rf)
print(f"The optimal weight allocation is: Asset 1: {w1:.4f}, Asset 2: {w2:.4f}")
# so that the portfolio has the highest Sharpe ratio
rp = w1 * r1 + w2 * r2
stdp = ((w1 * std1)**2 + (w2 * std2)**2 + 2 * w1 * w2 * std1 * std2 * rho)**0.5
sharpe_ratio = (rp - rf) / stdp
print("expected return of the portfolio: ", rp)
print("standard deviation of the portfolio: ", stdp)
print("Sharpe ratio: ", sharpe_ratio)

The optimal weight allocation is: Asset 1: 0.8403, Asset 2: 0.1597
expected return of the portfolio:  0.051927994466421275
standard deviation of the portfolio:  0.039909501477286426
Sharpe ratio:  0.8000098544100422


In [69]:
# given return needed, calculate the optimal weight allocation between rf and rp
return_needed = 0.04
w_rf = (return_needed - rp)/(rf - rp)
w_rp = 1 - w_rf
print(f"The optimal weight allocation is: rf: {w_rf:.4f}, rp: {w_rp:.4f}")
w1_final = w1 * w_rp
w2_final = w2 * w_rp
print(f"The final weight allocation is: rf: {w_rf:.4f}, Asset 1: {w1_final:.4f}, Asset 2: {w2_final:.4f}")

The optimal weight allocation is: rf: 0.3736, rp: 0.6264
The final weight allocation is: rf: 0.3736, Asset 1: 0.5263, Asset 2: 0.1001


# CAPM

Assumptions about CAMP:

Individual:
1. investors are rational and risk-averse mean-variance optimizer
2. Investors have single-period common planning horizon
3. investors have homogeneous expections about expected join distribution of returns (no information asymmetry)

Market:
1. All assets are publicly held and trade on public exchanges
2. investors and shor sell
3. no tax and transaction costs

capital market line: allocation of the market portfolio and risk-free assets,
$$
    E(R_s) = R_f + \frac{E(R_M)-R_f}{\sigma_M}\sigma_e
$$


CAPM: Capital Asset Pricing Model, it depicts a security market line by modifying $\beta$, where $\beta =\frac{\sigma_{i,M}}{\sigma_M^2} $

$$
    E(r_i) = r_f + \frac{\sigma_{i,M}}{\sigma_M^2}(E(r_M) - r_f)
$$


Since CAPM is calculated based on single-period common planning horizon, building on the ex-ante beta. However, we cannot know future information. We need to estimate it via historical proxies.

$$
(R(r_i)-r_f )_t = \alpha_i + \beta_i ( E(r_M) - r_f )_t + \epsilon_{i,t}
$$

- $ \alpha $ = Intercept term representing the risk-adjusted return
- $ \beta $ = Beta, a measure of the asset's systematic risk relative to the market (excess return to variation in market)
- $ r_M $ = Return on the market portfolio
- $ \epsilon $ = Error term capturing firm-specific risks, with $ E(\epsilon) = 0 $

Ideally, in equalibrium, $\alpha$ should be 0, but it is NOT!

1. if CAPM is false, a positive alpha represents a risk-premium being earned beyond the market premium factor.
2. if CAPM is true, the alpha simply capturing the excess returns due to systematic risks that the proxy cannot capture.
